# Mathematical Underpinnings - Lab 6

In [2]:
from sklearn.metrics import mutual_info_score
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from tqdm import tqdm

## Useful functions

In [3]:
def discetize_2bins(X):
    X_discrete = 1*(X >= 0)
    return X_discrete

In [4]:
def conditional_permutation(X, Z):

    z_values = np.unique(Z)
    n_z_values = len(z_values)
    n = len(Z)

    X_b = np.zeros(n)

    for i in range(n_z_values):

        z_value_tmp = z_values[i]

        X_b[Z == z_value_tmp] = np.random.permutation(X[Z == z_value_tmp])

    return X_b

In [5]:
def conditional_mutual_information(X, Y, Z):

    z_values = np.unique(Z)
    n_z_values = len(z_values)
    n = len(Z)

    cmi = 0

    for i in range(n_z_values):

        z_value_tmp = z_values[i]
        z_condition = (Z == z_value_tmp)

        X_z = X[z_condition]
        Y_z = Y[z_condition]

        mi_XY_z = mutual_info_score(X_z, Y_z)
        p_z = np.sum(z_condition)/n

        cmi += p_z*mi_XY_z

    return cmi

In [6]:
# II(X;Y;Z)
def interaction_information(X, Y, Z):
    return conditional_mutual_information(X, Y, Z) - mutual_info_score(X, Y)

In [7]:
# II(X;Y;Z1;Z2)
def interaction_information2(X, Y, Z1, Z2):
    Z_1_and_2 = 2*Z2 + Z1
    return interaction_information(X, Y, Z_1_and_2) - interaction_information(X, Y, Z1) - interaction_information(X, Y, Z2)

## Task 1

In [8]:
def secmi2(X, Y, Z):
    return sum(
        [
            interaction_information(X, Y, Z[:, i])
            for i in range(Z.shape[1])
        ],
        start=mutual_info_score(X, Y)
    )

def secmi3(X, Y, Z):
    return sum(
        [
            interaction_information2(X, Y, Z[:, i], Z[:, j])
            for i in range(Z.shape[1])
            for j in range(i)
        ],
        start=secmi2(X, Y, Z)
    )

### a)

In [9]:
def cond_indep_test_permutation(X, Y, Z, B, stat):
    n_col_Z = Z.shape[1]
    Z_1dim = np.dot(Z, 2**np.linspace(0, n_col_Z-1, n_col_Z))

    if stat == "cmi":
        stat_value = conditional_mutual_information(X, Y, Z_1dim)
    if stat == "secmi2":
        stat_value = secmi2(X, Y, Z)
    if stat == "secmi3":
        stat_value = secmi3(X, Y, Z)

    condition_p_value = 0
    for b in range(B):
        X_b = conditional_permutation(X, Z_1dim)

        if stat == "cmi":
            stat_value_b = conditional_mutual_information(X_b, Y, Z_1dim)
        if stat == "secmi2":
            stat_value_b = secmi2(X_b, Y, Z)
        if stat == "secmi3":
            stat_value_b = secmi3(X_b, Y, Z)

        if stat_value <= stat_value_b:
            condition_p_value += 1

    p_value = (1 + condition_p_value)/(1 + B)

    return 2*len(X)*stat_value, p_value

### b)

In [14]:
def sample_data_1(n=1000):
    Y = discetize_2bins(np.random.standard_normal(n))
    Z = discetize_2bins(
        np.random.standard_normal(n * 3).reshape((n, 3)) + Y.reshape((n, 1))
    )
    X = discetize_2bins(np.random.standard_normal(n) + Z[:, 0])
    return X, Y, Z

results_1 = []
for trial in tqdm(range(100)):
    X, Y, Z = sample_data_1()
    for features, indices in {"z1, z2": [0, 1], "z2, z3": [1, 2]}.items():
        for stat in ["cmi", "secmi2", "secmi3"]:
            _, p_value = cond_indep_test_permutation(X, Y, Z[:, indices], B=50, stat=stat)
            results_1.append(
                {"trial": trial, "features": features, "stat": stat, "p_value": p_value}
            )
results_1 = pd.DataFrame(results_1)
results_1['rejected'] = results_1['p_value'] < 0.05
results_1.groupby(["features", "stat"]).agg({"rejected": "sum"})

100%|██████████| 100/100 [08:10<00:00,  4.91s/it]


rejected
features stat            
z1, z2   cmi            2
         secmi2         4
         secmi3         3
z2, z3   cmi           80
         secmi2        78
         secmi3        75

### c)

In [16]:
def sample_data_2(n=1000):
    X = np.random.binomial(1, 0.5, n)
    Z = np.random.binomial(1, 0.5, n * 3).reshape(n, 3)
    Y = np.random.binomial(
        1, np.where((X + Z[:, 0] + Z[:, 1]) % 2 == 1, 0.8, 0.2), n
    )
    return X, Y, Z

results_2 = []
for trial in tqdm(range(100)):
    X, Y, Z = sample_data_2()
    for features, indices in {"z1, z2": [0, 1], "z2, z3": [1, 2]}.items():
        for stat in ["cmi", "secmi2", "secmi3"]:
            _, p_value = cond_indep_test_permutation(X, Y, Z[:, indices], B=50, stat=stat)
            results_2.append(
                {"trial": trial, "features": features, "stat": stat, "p_value": p_value}
            )
results_2 = pd.DataFrame(results_2)
results_2['rejected'] = results_2['p_value'] < 0.05
results_2.groupby(["features", "stat"]).agg({"rejected": "sum"})

100%|██████████| 100/100 [08:05<00:00,  4.85s/it]


rejected
features stat            
z1, z2   cmi          100
         secmi2         6
         secmi3       100
z2, z3   cmi            3
         secmi2         4
         secmi3         4

## Task 2

in R